<a href="https://colab.research.google.com/github/yunusemretom/xtts-finetune-webui/blob/main/notebook/xtts_finetune_webui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Welcome to the *xtts*-finetune-webui gradio gui!

This webui is a slightly modified copy of the official webui for finetune xtts.

If you are looking for an option for normal XTTS use look here https://github.com/daswer123/xtts-webui

In [1]:
# @title 🛠️ Install requirments
#!DEBIAN_FRONTEND=noninteractive
!sudo apt-get update # && sudo apt-get -y upgrade
!sudo apt-get -y install libegl1
!sudo apt-get -y install libopengl0
!sudo apt-get -y install libxcb-cursor0
!pip install -r https://raw.githubusercontent.com/daswer123/xtts-finetune-webui/main/requirements.txt
!pip install gradio==4.44.1
!pip install fastapi==0.103.1
!pip install pydantic==2.3.0

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,165 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,425 kB]
Get:13 https://developer.download.nvidia.com/compute/c

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 26.4 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.4.0
    Uninstalling gradio_client-1.4.0:
      Successfully uninstalled gradio_client-1.4.0
  Attempting uninstall: gradio
    Found existing installation: gradio 5.1.0
    Uninstalling gradio-5.1.0:
      Successfully uninstalled gradio-5.1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: starlette
    Found existing installation: starlette 0.41.2
    Uninstalling starlette-0.41.2:
      Successfully uninstalled starlette-0.41.2
  Attempting uninstall: fastapi
    Found existing installation: fastapi 0.115.5
    Uninstalling fastapi-0.115.5:
      Successfully uninstalled fastapi-0.115.5
     ━━━━━━━━━━━━━━━━━━

In [ ]:
# @title 🚀 Run interface
%cd /content/
!git clone https://github.com/DrewThomasson/xtts-finetune-webui.git
%cd /content/xtts-finetune-webui
!python xtts_demo.py --share

/content
Cloning into 'xtts-finetune-webui'...
remote: Enumerating objects: 202, done.
remote: Counting objects: 100% (139/139), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 202 (delta 90), reused 99 (delta 64), pack-reused 63 (from 1)
Receiving objects: 100% (202/202), 71.64 KiB | 2.87 MiB/s, done.
Resolving deltas: 100% (105/105), done.
/content/xtts-finetune-webui
Running on local URL:  http://127.0.0.1:5003
Running on public URL: https://d876cd7e9bfd8a6a3c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
config.json: 100% 2.26k/2.26k [00:00<00:00, 11.9MB/s]
tokenizer.json:   0% 0.00/2.20M [00:00<?, ?B/s]
vocabulary.txt:   0% 0.00/460k [00:00<?, ?B/s]

vocabulary.txt: 100% 460k/460k [00:00<00:00, 7.45MB/s]
tokenizer.json: 100% 2.20M/2.20M [00:00<00:00, 22.5MB/s]


model.bin:   1% 10.5M/1.53G [00:00<00:39, 38.4MB/s]

model.bin:   

In [ ]:
!pip install git+https://github.com/Hasan-Naseer/whisperX.git@release/latest-faster-whisper-version
!pip install ctranslate2==4.4.0

In [ ]:
import shutil
import requests
import os
from tqdm import tqdm  # Progress bar library

# Define the paths
finetune_dir = '/content/xtts-finetune-webui/finetune_models/ready'  # @param {type:"string"}
dataset_dir = '/content/xtts-finetune-webui/finetune_models/dataset'  # @param {type:"string"}

# Create a temporary directory to collect both folders before zipping
temp_dir = "/content/temp_finetune_dataset"
os.makedirs(temp_dir, exist_ok=True)

# Copy both directories into the temporary directory with a progress bar
def copy_with_progress(src, dst):
    total_files = sum(len(files) for _, _, files in os.walk(src))
    with tqdm(total=total_files, desc=f"Copying {os.path.basename(src)}") as pbar:
        for root, _, files in os.walk(src):
            rel_path = os.path.relpath(root, src)
            target_path = os.path.join(dst, rel_path)
            os.makedirs(target_path, exist_ok=True)
            for file in files:
                shutil.copy(os.path.join(root, file), target_path)
                pbar.update(1)

copy_with_progress(finetune_dir, os.path.join(temp_dir, "ready"))
copy_with_progress(dataset_dir, os.path.join(temp_dir, "dataset"))

# Create a zip file of the combined directories with progress
zip_filename = "finetune_and_dataset.zip"
with tqdm(total=100, desc="Zipping files") as pbar:
    shutil.make_archive("finetune_and_dataset", 'zip', root_dir=temp_dir)
    pbar.update(100)

# Define a function to stream the upload with a progress bar
def upload_with_progress(file_path, url):
    file_size = os.path.getsize(file_path)
    with open(file_path, 'rb') as f, tqdm(
        total=file_size, unit='B', unit_scale=True, desc="Uploading"
    ) as progress:
        response = requests.post(
            url,
            files={"file": (file_path, f)},
            stream=True,
            headers={"Connection": "keep-alive"},
        )
        # Update the progress bar as chunks are sent
        for chunk in response.iter_content(chunk_size=4096):
            if chunk:
                progress.update(len(chunk))
    return response

# Upload the zip file to file.io with a progress bar
response = upload_with_progress(zip_filename, "https://file.io/?expires=1d")

# Parse the response and display the download link
if response.status_code == 200:
    download_link = response.json().get('link', 'Error: No link found.')
    print(f"Your file is ready: {download_link}")
else:
    print(f"Failed to upload: {response.status_code} - {response.text}")
